In [25]:
import sys
import pprint
import spotipy
import spotipy.util as util
from urllib.request import urlopen
from spotipy.oauth2 import SpotifyClientCredentials
from PIL import Image
from IPython.core.display import display, HTML
from functools import reduce
from collections import Counter
import pymongo
from pymongo import MongoClient
from bson.objectid import ObjectId

## SETUP
pp = pprint.PrettyPrinter()
client = MongoClient('localhost', 27017)
db = client.server

client_credentials_manager = SpotifyClientCredentials(
    client_id='9c93bd032a4340b086b31bd30ec8f24c',
    client_secret='b893fd0a51d34e2399effa91c1026de7')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

## AUX FUNCTIONS
def printTrackSimple(track):
    print('(' + track['id'] + ') - ' + track['name'] + ' - ' + track['artists'][0]['name'])    
    return

def printTrackFull(track):
    feature_raw = [feature for feature in features_list if feature['id'] == track['id']][0]
    feature = dict((key, feature_raw[key]) for key in (
        'acousticness',
        'danceability',
        'energy',
        'instrumentalness',
        'liveness',
        'speechiness',
        'valence'))
    feature_meta = dict((key, feature_raw[key]) for key in (
        'duration_ms',
        'id',
        'key',
        'loudness',
        'mode',
        'tempo',
        'time_signature'))
    display(HTML("""
         <table style='width: 100%;padding: 0px; margin: 0px'>
            <tr>
                <td style='width: 64px;padding: 0px'>
                    <img src='"""+track['album']['images'][-1]['url']+"""'>
                </td>
                <td style='text-align: left;width: 300px'>
                    <b>"""+track['name']+"""</b>
                    <br>
                    <font color='grey'>"""+track['artists'][0]['name']+"""</font>
                </td>""" +
                 ''.join([str("""<td style='text-align: left;writing-mode: tb-rl;transform: rotate(-180deg)'>
                    <div style="height:64px; overflow:hidden">
                        """)+str(feature_name)+str("""
                        <br>
                        <font color='darkgrey'>""") + str(feature_meta[feature_name]) + str("""</font>                        
                    </div>
                </td>""") for feature_name in feature_meta]) + 
                 ''.join([str("""<td style='text-align: left;writing-mode: tb-rl;transform: rotate(-180deg)'>
                    <div style="height:64px; overflow:hidden">
                        """)+str(feature_name)+str("""
                        <br>
                        <progress value='""") + str(feature[feature_name]) + str("""' max='1' style='height: 64px;transform: rotate(180deg);'></progress>                        
                    </div>
                </td>""") for feature_name in feature]) +
"""            </tr>
        </table>"""))
    return

In [26]:
## CODE
username = 'Ariel Godinho'
scope = 'user-top-read user-library-read'
max_songs = 5000 #00
token = util.prompt_for_user_token(
    username,
    scope,
    client_id='9c93bd032a4340b086b31bd30ec8f24c',
    client_secret='b893fd0a51d34e2399effa91c1026de7',
    redirect_uri='http://localhost:8888/tree')

if token:
    #Set token to spotify library
    sp = spotipy.Spotify(auth=token)
    print('Loading saved tracks...')
    user_tracks = []
    for offset in range(0, max_songs, 50):
        results = sp.current_user_saved_tracks(50,offset)
        if(offset > results['total']):
            break
        user_tracks.extend([item['track'] for item in results['items']])
        print(len(user_tracks), end='\r')    
    print(len(user_tracks))
    
    #print('Loading tracks features...')
    #features_list = []
    #for index in range(0, len(user_tracks), 100):
    #    features_list.extend(sp.audio_features(tracks=[track['id'] for track in user_tracks[index:index+100]]))
    #    print(len(features_list), end='\r')   
    #print(len(features_list))
    
else:
    print("Can't get token for", username)

Loading saved tracks...
1473


In [215]:
## DISCOVER NEW TRACKS (based on user's profile)

## ARTISTS
# Get all user's artists
all_user_artists = []
[[all_user_artists.append(artist['id'])
    for artist in artist_list]
        for artist_list in [track['artists'] for track in user_tracks]]
all_user_artists = set(all_user_artists)
print("User's unique artists: ", len(all_user_artists))

# Get related artists
print('Loading related artists...')
all_related_artists = []
for artist_id in all_user_artists:
    related_artists = sp.artist_related_artists(artist_id)
    all_related_artists.extend([artist['id'] for artist in related_artists['artists']])
    print(len(all_related_artists), end='\r')
print(len(all_related_artists))
all_related_artists = set(all_related_artists)
print("User's unique related artists: ", len(all_related_artists))

User's unique artists:  1167
Loading related artists...
retrying ...1secs
22354
User's unique related artists:  9514


In [232]:
# Get all artist's top tracks
print('Loading artists tracks...')
#all_tracks = []
#i = 0
for artist_id in list(all_related_artists)[2940:]:
    i += 1
    artist_top_tracks = sp.artist_top_tracks(artist_id)
    #all_tracks.extend([(track['artists'][0]['name'], track['name'], track['id']) for track in artist_top_tracks['tracks']])
    all_tracks.extend(artist_top_tracks['tracks'])
    print('('+str(i)+'/'+str(len(all_related_artists))+')', len(all_tracks), end='\r')
print('('+str(i)+'/'+str(len(all_related_artists))+')', len(all_tracks))
#all_tracks = set(all_tracks)
print("Unique tracks: ", len(all_tracks))
## ALBUMS
# Get top albums

# Get top track's albums

# Get album's tracks

## FEATURES
# Get features


Loading artists tracks...
retrying ...1secs
retrying ...1secs 60112
retrying ...1secs 61267 61527 65297 66110 67763
(9516/9514) 87300 75644 7682281689
Unique tracks:  87300


In [279]:
## MONGODB
for track in user_tracks:
    track_item = {
        'id': track['id'],
        'album_id': track['album']['id'],
        'artists_ids': [artist['id'] for artist in track['artists']],
        'disc_number': track['disc_number'],
        'duration_ms': track['duration_ms'],
        'explicit': track['explicit'],
        'external_ids': track['external_ids'],
        'external_urls': track['external_urls'],
        'href': track['href'],
        'is_local': track['is_local'],
        'name': track['name'],
        'popularity': track['popularity'],
        'preview_url': track['preview_url'],
        'track_number': track['track_number'],
        'type': track['type'],
        'uri': track['uri'],
    }
    db.tracks.update_one({'id' : track_item['id']}, {"$set": track_item}, upsert=True)

In [16]:
tracks_ids = [track['id'] for track in user_tracks]
document = {
    'id': sp.current_user()['id'],
    'tracks_ids': tracks_ids
}
db.users.insert_one(document)

In [23]:
user = db.users.find_one({'id' : '12152580425'})
pp.pprint(user)

{'_id': ObjectId('5bf030bbb1aafa2c946aa860'),
 'id': '12152580425',
 'tracks_ids': ['4gEFeIEORz8thfqMY7nKcb',
                '6zQTiw3M1p58eklHhCBWyi',
                '0vWNGOcd15MecWZIw0jKcE',
                '0xJLcjd0gaZct43xG1UlXS',
                '7xyYsOvq5Ec3P4fr6mM9fD',
                '7ouMYWpwJ422jRcDASZB7P',
                '3Sno9FE8r2uz8QP0MtnTPL',
                '5YXr4AGfUQpLSxtFSsKUh6',
                '3lPr8ghNDBLc2uZovNyLs9',
                '3skn2lauGk7Dx6bVIt5DVj',
                '5D5HuZt3z1y7JSoWPCZfF7',
                '114rzL6VEy9bb3amPcY3tw',
                '1pbyc2rbpwxZjCbKC2X7UV',
                '28FJMlLUu9NHuwlZWFKDn7',
                '1gJFn507Rrd9MmngICj02L',
                '0tHbQRjL5phd8OoYl2Bdnd',
                '0It6VJoMAare1zdV2wxqZq',
                '1C2QJNTmsTxCDBuIgai8QV',
                '4VqPOruhp5EdPBeR92t6lQ',
                '5N5k9nd479b1xpDZ4usjrg',
                '6QgjcU0zLnzq5OrUoSZ3OK',
                '6znZbTV31rdQmVDocXV8vH',
        

                '1RKUoGiLEbcXN4GY4spQDx',
                '0d28khcov6AiegSCpG5TuT',
                '1vxw6aYJls2oq3gW0DujAo',
                '5XUyysqMPqWDF2KIbbRvKT',
                '6KqKg8IPuvtDB3PNAvffFf',
                '6lV2MSQmRIkycDScNtrBXO',
                '5pBvNeOAJ54zgd5lEOmM3b',
                '4hEl2wrneFEJbJ6SDYWd2f',
                '0mwXfPCm4AMaz9J0Cn7FXJ',
                '1MYlx4dBtiyjn7K8YSyfzT',
                '06aw4JI5nYa8XBeBKVrltZ',
                '17baAghWcrewNOcc9dCewx',
                '7w9bgPAmPTtrkt2v16QWvQ',
                '6sDQ4uiWw9OdVrCXFLSlZt',
                '6yr8GiTHWvFfi4o6Q5ebdT',
                '6UaRii9AH6Zss9xNMEQ2M9',
                '5U8hKxSaDXB8cVeLFQjvwx',
                '3U6kC4Ko4VT1ZvEKjEZsC1',
                '3FrX3mx8qq7SZX2NYbzDoj',
                '7ks6AZmFcm3Y6PGGxGSmlB',
                '0TBaFEx7WENZgqLXWErD0b',
                '0yl1LvI69CJrQlXJoYnU97',
                '0k1xMUwn9sb7bZiqdT9ygx',
                '0IedgQjjJ8Ad4B3UD

In [326]:
for item in db.tracks.find({'features' : None}):
    print(item['id'], item['name'])

1uG7rMgZ4GcrUF5oV7x1St Heartbeat
2RRoTAmLlwDOPVJQRC2xHe Catch Me If I Fall
6iufHPrCfMmeEm85L2hl7Z Waiting For Winter (ABGT303) - No Mana Remix
6ziS59yxSUNV34WYcaiDs1 Transit of Venus (ABGT303)
5eFO753r5qvS5E8riE4kdg Stay (ABGT303)
7rF8GjLZr8bzFuul4XJUev Dollar For Your Sadness
6iGjcXakhZuScYYuGP0piO Starstruck - Alternate Mix With Session Chat
1yhb1S2IIQppi7Xp25AQX1 All I Want for Christmas Is You
3nj8I6aGKFBtYwDBI1CyIf Love Spy - Instrumental
5CmMyiq4um2aFhiLZ1rPWo Best of 2016 (Album Mix)
2SwDlFKj3wjuNEr0LL6yzr Lights - WildVibes Remix
2SwDlFKj3wjuNEr0LL6yzr Lights - WildVibes Remix


In [302]:
pp.pprint(db.tracks.find_one({'id' : '0FwXYnBXPFaHyeG7EOjVt5'}))

{'_id': ObjectId('5bd67745edae5e837b6448c4'),
 'album_id': '1V7tju2mbF2q6bZZVV36mM',
 'artists_ids': ['57gnEMExR8CJdZ7MYYbbrN'],
 'disc_number': 1,
 'duration_ms': 237888,
 'explicit': False,
 'external_ids': {'isrc': 'CABD61604902'},
 'external_urls': {'spotify': 'https://open.spotify.com/track/0FwXYnBXPFaHyeG7EOjVt5'},
 'features': {'acousticness': 0.045,
              'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0FwXYnBXPFaHyeG7EOjVt5',
              'danceability': 0.678,
              'duration_ms': 237888,
              'energy': 0.868,
              'id': '0FwXYnBXPFaHyeG7EOjVt5',
              'instrumentalness': 0.0227,
              'key': 0,
              'liveness': 0.0749,
              'loudness': -4.987,
              'mode': 1,
              'speechiness': 0.0493,
              'tempo': 92.019,
              'time_signature': 4,
              'track_href': 'https://api.spotify.com/v1/tracks/0FwXYnBXPFaHyeG7EOjVt5',
              'type': 'audio_features',
 

In [285]:
db.tracks.count_documents({})

85510

In [243]:
print('Loading tracks features...')
for index in range(0, len(all_tracks), 100):
    features_list.extend(sp.audio_features(tracks=[track['id'] for track in all_tracks[index:index+100]]))
    print(len(features_list), end='\r')   
print(len(features_list))

Loading tracks features...
88772


In [321]:
## MONGODB
print('Inserting tracks features...')
i = 0
j = 0
for features in features_list:
    i += 1
    #pp.pprint(features)
    if(features is not None):
        bla = db.tracks.update_one({'id' : features['id']}, {"$set": {'features': features}})
    else:
        j += 1
    print(i, end='\r')   
print(i)
print('Tracks without features: ', j)

Inserting tracks features...
88772
Tracks without features:  10


In [317]:
pp.pprint(features_list[1485:1500])

[{'acousticness': 0.0159,
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4sBuUaRokHP5NZao9t559V',
  'danceability': 0.659,
  'duration_ms': 218638,
  'energy': 0.87,
  'id': '4sBuUaRokHP5NZao9t559V',
  'instrumentalness': 0.28,
  'key': 10,
  'liveness': 0.123,
  'loudness': -4.904,
  'mode': 1,
  'speechiness': 0.0331,
  'tempo': 124.003,
  'time_signature': 4,
  'track_href': 'https://api.spotify.com/v1/tracks/4sBuUaRokHP5NZao9t559V',
  'type': 'audio_features',
  'uri': 'spotify:track:4sBuUaRokHP5NZao9t559V',
  'valence': 0.44},
 {'acousticness': 0.173,
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5deRG3zaTg5qzFe5Jm30vb',
  'danceability': 0.757,
  'duration_ms': 192317,
  'energy': 0.599,
  'id': '5deRG3zaTg5qzFe5Jm30vb',
  'instrumentalness': 0.00244,
  'key': 6,
  'liveness': 0.0847,
  'loudness': -5.081,
  'mode': 1,
  'speechiness': 0.0293,
  'tempo': 140.053,
  'time_signature': 4,
  'track_href': 'https://api.spotify.com/v1/tracks/5deRG3zaTg5qzFe

In [141]:
# Full print
for track in user_tracks[0:10]:
    printTrackFull(track)

,Coisas Que Eu Sei Danni Carlos,duration_ms 232093,id 6zQTiw3M1p58eklHhCBWyi,key 8,loudness -14.312,mode 0,tempo 119.966,time_signature 4,acousticness,danceability,energy,instrumentalness,liveness,speechiness,valence


,Serei Luz Natiruts,duration_ms 307061,id 0vWNGOcd15MecWZIw0jKcE,key 0,loudness -6.87,mode 0,tempo 169.982,time_signature 4,acousticness,danceability,energy,instrumentalness,liveness,speechiness,valence


,Butterflies and Hurricanes Muse,duration_ms 301733,id 0xJLcjd0gaZct43xG1UlXS,key 9,loudness -8.56,mode 1,tempo 115.326,time_signature 4,acousticness,danceability,energy,instrumentalness,liveness,speechiness,valence


,Hysteria Muse,duration_ms 227440,id 7xyYsOvq5Ec3P4fr6mM9fD,key 9,loudness -4.852,mode 0,tempo 93.162,time_signature 4,acousticness,danceability,energy,instrumentalness,liveness,speechiness,valence


,Knights Of Cydonia Muse,duration_ms 366213,id 7ouMYWpwJ422jRcDASZB7P,key 11,loudness -5.301,mode 0,tempo 137.114,time_signature 4,acousticness,danceability,energy,instrumentalness,liveness,speechiness,valence


,City Of Delusion Muse,duration_ms 288266,id 3Sno9FE8r2uz8QP0MtnTPL,key 2,loudness -5.158,mode 0,tempo 119.817,time_signature 4,acousticness,danceability,energy,instrumentalness,liveness,speechiness,valence


,Map Of The Problematique Muse,duration_ms 258066,id 5YXr4AGfUQpLSxtFSsKUh6,key 8,loudness -3.613,mode 1,tempo 125.04,time_signature 4,acousticness,danceability,energy,instrumentalness,liveness,speechiness,valence


,Supermassive Black Hole Muse,duration_ms 212440,id 3lPr8ghNDBLc2uZovNyLs9,key 7,loudness -3.727,mode 1,tempo 120.0,time_signature 4,acousticness,danceability,energy,instrumentalness,liveness,speechiness,valence


,Starlight Muse,duration_ms 240213,id 3skn2lauGk7Dx6bVIt5DVj,key 4,loudness -4.046,mode 1,tempo 121.61,time_signature 4,acousticness,danceability,energy,instrumentalness,liveness,speechiness,valence


,Dentro da Música II - Ao Vivo Natiruts,duration_ms 310400,id 5D5HuZt3z1y7JSoWPCZfF7,key 7,loudness -8.203,mode 0,tempo 103.976,time_signature 4,acousticness,danceability,energy,instrumentalness,liveness,speechiness,valence


In [ ]:
# Simple print
for track in user_tracks[0:10]:
    printTrackSimple(track)

In [5]:
# Extract ids
print([track['id'] for track in user_tracks[0:10]])

['6zQTiw3M1p58eklHhCBWyi', '0vWNGOcd15MecWZIw0jKcE', '0xJLcjd0gaZct43xG1UlXS', '7xyYsOvq5Ec3P4fr6mM9fD', '7ouMYWpwJ422jRcDASZB7P', '3Sno9FE8r2uz8QP0MtnTPL', '5YXr4AGfUQpLSxtFSsKUh6', '3lPr8ghNDBLc2uZovNyLs9', '3skn2lauGk7Dx6bVIt5DVj', '5D5HuZt3z1y7JSoWPCZfF7']


In [6]:
# Get user's top artists
top_artists = sp.current_user_top_artists(50, 0, 'long_term')
pp.pprint([artist['name'] for artist in top_artists['items']])

['Natiruts',
 'The Weeknd',
 'Daft Punk',
 'Years & Years',
 'Jorge & Mateus',
 'Drake',
 'Darren Korb',
 'Cacife Clandestino',
 'Maneva',
 'Muse',
 'Cidade Negra',
 'Djavan',
 'Disclosure',
 'Avicii',
 'Eminem',
 'Kamelot',
 'Calvin Harris',
 'Criolo',
 'Seu Jorge',
 'Red Hot Chili Peppers',
 'Os Paralamas Do Sucesso',
 'Lulu Santos',
 'Galantis',
 'Angra',
 'Major Lazer',
 'Black Sabbath',
 'Cássia Eller',
 'Bob Marley & The Wailers',
 'Projota',
 'David Guetta',
 'Incubus',
 'Symphony X',
 'Gorillaz',
 'Tiësto',
 'Michael Jackson',
 'Arctic Monkeys',
 'Ed Sheeran',
 'Queen',
 'Vicetone',
 'Maroon 5',
 'Nirvana',
 'Emicida',
 'Audioslave',
 'Zedd',
 'Justin Timberlake',
 'Tove Lo',
 'Charlie Brown Jr.',
 'Armin van Buuren',
 'Pink Floyd',
 'Michael Bublé']


In [ ]:
# Get user's top tracks
top_tracks = sp.current_user_top_tracks(50, 0, 'long_term')
pp.pprint([track['name'] for track in top_tracks['items']])

In [ ]:
# Get spotify categories
categories = sp.categories(limit=50, offset=0)
pp.pprint([category['name'] for category in categories['categories']['items']])

In [60]:
# Get related artist from artist id
related_artists = sp.artist_related_artists('06cd30Cv9US973Ika84gDw')
pp.pprint([artist['name'] for artist in related_artists['artists']])

['Israel Novaes',
 'Maiara & Maraisa',
 'Cleber & Cauan',
 'Pedro Paulo & Alex',
 'George Henrique & Rodrigo',
 'Cristiano Araújo',
 'Edu Chociay',
 'Naiara Azevedo',
 'Felipe Araújo',
 'Jefferson Moraes',
 'Diego & Victor Hugo',
 'Henrique & Diego',
 'Bruninho & Davi',
 'Thaeme & Thiago',
 'Zé Neto & Cristiano',
 'Israel & Rodolffo',
 'Fernando & Sorocaba',
 'Marcos & Belutti',
 'Paula Mattos',
 'Gabriel Gava']


In [78]:
# Get artist's top tracks from artist id
artist_top_tracks = sp.artist_top_tracks('4PhdYoQGH8s1xee81dqQOO')
pp.pprint([track for track in artist_top_tracks['tracks']])

[{'album': {'album_type': 'album',
            'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4PhdYoQGH8s1xee81dqQOO'},
                         'href': 'https://api.spotify.com/v1/artists/4PhdYoQGH8s1xee81dqQOO',
                         'id': '4PhdYoQGH8s1xee81dqQOO',
                         'name': 'Natiruts',
                         'type': 'artist',
                         'uri': 'spotify:artist:4PhdYoQGH8s1xee81dqQOO'}],
            'external_urls': {'spotify': 'https://open.spotify.com/album/14iEAaKYHYBx8jMEtzKmSX'},
            'href': 'https://api.spotify.com/v1/albums/14iEAaKYHYBx8jMEtzKmSX',
            'id': '14iEAaKYHYBx8jMEtzKmSX',
            'images': [{'height': 640,
                        'url': 'https://i.scdn.co/image/5130daae4b1c8e83ec10788deac0a9ce16fc0521',
                        'width': 640},
                       {'height': 300,
                        'url': 'https://i.scdn.co/image/99eb31e5365ba0446b1ea349495ac88d69174d32',


                        'width': 300},
                       {'height': 64,
                        'url': 'https://i.scdn.co/image/84e935d0925d259112e9b8e6cf1256a2ca05dfaf',
                        'width': 64}],
            'name': 'Raçaman',
            'release_date': '2009',
            'release_date_precision': 'year',
            'total_tracks': 12,
            'type': 'album',
            'uri': 'spotify:album:7dlYoL0Bm3AJXTlzaKwzMr'},
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4PhdYoQGH8s1xee81dqQOO'},
               'href': 'https://api.spotify.com/v1/artists/4PhdYoQGH8s1xee81dqQOO',
               'id': '4PhdYoQGH8s1xee81dqQOO',
               'name': 'Natiruts',
               'type': 'artist',
               'uri': 'spotify:artist:4PhdYoQGH8s1xee81dqQOO'}],
  'disc_number': 1,
  'duration_ms': 332653,
  'explicit': False,
  'external_ids': {'isrc': 'BRXCR0900005'},
  'external_urls': {'spotify': 'https://open.spotify.com/track/5lKHrBigpyK

               'uri': 'spotify:artist:4PhdYoQGH8s1xee81dqQOO'}],
  'disc_number': 1,
  'duration_ms': 258166,
  'explicit': False,
  'external_ids': {'isrc': 'BRSME1701360'},
  'external_urls': {'spotify': 'https://open.spotify.com/track/3KSj0znpvfGKrBzEoE8gcL'},
  'href': 'https://api.spotify.com/v1/tracks/3KSj0znpvfGKrBzEoE8gcL',
  'id': '3KSj0znpvfGKrBzEoE8gcL',
  'is_local': False,
  'is_playable': True,
  'name': 'Na Positiva',
  'popularity': 58,
  'preview_url': 'https://p.scdn.co/mp3-preview/ac041252a450659eb04e3892cf608f26f1ace27f?cid=9c93bd032a4340b086b31bd30ec8f24c',
  'track_number': 1,
  'type': 'track',
  'uri': 'spotify:track:3KSj0znpvfGKrBzEoE8gcL'}]


In [28]:
# Get spotify available genre seeds
genre_seeds = sp.recommendation_genre_seeds()['genres']
pp.pprint(genre_seeds)

['acoustic',
 'afrobeat',
 'alt-rock',
 'alternative',
 'ambient',
 'anime',
 'black-metal',
 'bluegrass',
 'blues',
 'bossanova',
 'brazil',
 'breakbeat',
 'british',
 'cantopop',
 'chicago-house',
 'children',
 'chill',
 'classical',
 'club',
 'comedy',
 'country',
 'dance',
 'dancehall',
 'death-metal',
 'deep-house',
 'detroit-techno',
 'disco',
 'disney',
 'drum-and-bass',
 'dub',
 'dubstep',
 'edm',
 'electro',
 'electronic',
 'emo',
 'folk',
 'forro',
 'french',
 'funk',
 'garage',
 'german',
 'gospel',
 'goth',
 'grindcore',
 'groove',
 'grunge',
 'guitar',
 'happy',
 'hard-rock',
 'hardcore',
 'hardstyle',
 'heavy-metal',
 'hip-hop',
 'holidays',
 'honky-tonk',
 'house',
 'idm',
 'indian',
 'indie',
 'indie-pop',
 'industrial',
 'iranian',
 'j-dance',
 'j-idol',
 'j-pop',
 'j-rock',
 'jazz',
 'k-pop',
 'kids',
 'latin',
 'latino',
 'malay',
 'mandopop',
 'metal',
 'metal-misc',
 'metalcore',
 'minimal-techno',
 'movies',
 'mpb',
 'new-age',
 'new-release',
 'opera',
 'pagode',

In [29]:
# Extract user's top genres from user's top artists
top_artists = sp.current_user_top_artists(50, 0, 'medium_term')
top_genres = [artist['genres'] for artist in top_artists['items']]
top_genres_2d = reduce(lambda x, y: x+y, top_genres)
filtered_top_genres = [x for x in top_genres_2d if x in genre_seeds]
counter = Counter(filtered_top_genres)
common_genre_list = [genre[0] for genre in counter.most_common(5)]
pp.pprint(common_genre_list)

['rock', 'mpb', 'pop', 'samba', 'reggae']


In [39]:
# Get spotify recommendations from genre seeds
top_genre_count = 1
recomendation_list = sp.recommendations(seed_genres=common_genre_list[0:top_genre_count], limit=20)
pp.pprint([track['artists'][0]['name'] + ' - ' + track['name'] for track in recomendation_list['tracks']])

['Linkin Park - Faint',
 'Volbeat - Still Counting',
 "Simple Plan - I'm Just A Kid",
 'Queen - Radio Ga Ga - Remastered',
 'Seether - Fake It',
 'Iggy Pop - The Passenger',
 'All Time Low - Last Young Renegade',
 'ZZ Top - Sharp Dressed Man - 2008 Remastered Version',
 'Stone Sour - Taipei Person/Allah Tea',
 'Five Finger Death Punch - Trouble',
 "Bullet For My Valentine - Tears Don't Fall",
 'Broadside - Paradise',
 'Third Eye Blind - Semi-Charmed Life',
 "Aerosmith - I Don't Want to Miss a Thing - From the Touchstone film, "
 '"Armageddon"',
 'Extreme - More Than Words',
 'Danzig - Mother',
 'As I Lay Dying - My Own Grave',
 'K.Flay - High Enough',
 'Five Finger Death Punch - Sham Pain',
 'Motörhead - Ace of Spades']


In [37]:
for i in range(50):
    print(top_artists['items'][i]['name'], '-', top_genres[i])
#pp.pprint(top_genres)

Muse - ['modern rock', 'permanent wave', 'piano rock', 'post-grunge', 'rock']
Gorillaz - ['alternative hip hop', 'art pop']
Snarky Puppy - ['contemporary jazz', 'jazz funk', 'jazz fusion']
Greta Van Fleet - ['modern blues rock', 'modern hard rock', 'rock']
Natiruts - ['brazilian reggae']
Red Hot Chili Peppers - ['alternative rock', 'funk metal', 'permanent wave', 'post-grunge', 'rock']
The Weeknd - ['canadian pop', 'pop', 'rap']
Daft Punk - ['electro', 'electronic', 'filter house']
Drake - ['canadian hip hop', 'canadian pop', 'hip hop', 'pop rap', 'rap']
Darren Korb - ['chamber pop', 'deep chiptune', 'scorecore', 'video game music']
Cacife Clandestino - ['brazilian electronica', 'brazilian hip hop', 'deep funk carioca']
Eminem - ['detroit hip hop', 'g funk', 'hip hop', 'rap']
Led Zeppelin - ['album rock', 'classic rock', 'hard rock', 'rock']
Years & Years - ['dance pop', 'gauze pop', 'pop', 'tropical house']
Lulu Santos - ['brazilian rock', 'mpb', 'rock gaucho']
Saint Motel - ['indie p

In [38]:
pp.pprint(counter)

Counter({'rock': 11,
         'mpb': 10,
         'pop': 5,
         'samba': 4,
         'reggae': 3,
         'metal': 2,
         'pagode': 2,
         'edm': 2,
         'electro': 1,
         'electronic': 1,
         'dancehall': 1,
         'house': 1,
         'sertanejo': 1})
